In [1]:
#it's a library, so shhh!
shhh = function(...){
    suppressWarnings(
        suppressPackageStartupMessages(base::library(...))
    )
}

shhh(tidyverse)
shhh(psych)

In [4]:
#read in the data, skip the second row

readFile = readLines("./data/dat_cln.csv")
datFull = read.csv(textConnection(readFile[-2]), header = T, sep = ",")

Warning message in readLines("./data/dat_cln.csv"):
“incomplete final line found on './data/dat_cln.csv'”

In [18]:
datFull[, sort(names(datFull))]

EDU,HUM1,HUM10,HUM11_R,HUM12,HUM13_R,HUM14_R,HUM15_R,HUM16_R,HUM2_R,⋯,SA10,SA11,SA2,SA3,SA4,SA5,SA6,SA7,SA8,SA9
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
Doctoral degree,Neither agree nor disagree,Somewhat disagree,Neither agree nor disagree,Somewhat agree,Strongly agree,Neither agree nor disagree,Somewhat agree,Somewhat agree,Neither agree nor disagree,⋯,Somewhat disagree,Somewhat disagree,Strongly disagree,Strongly disagree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat disagree,Somewhat disagree,Strongly disagree
Master's degree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Strongly disagree,Somewhat agree,⋯,Somewhat disagree,Somewhat agree,Somewhat disagree,Strongly disagree,Somewhat agree,Somewhat agree,Somewhat agree,Neither agree nor disagree,Strongly agree,Strongly disagree
Doctoral degree,Strongly disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Strongly agree,Somewhat agree,Strongly agree,Somewhat agree,Somewhat agree,⋯,Somewhat agree,Somewhat disagree,Strongly disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Strongly disagree,Neither agree nor disagree,Somewhat disagree,Somewhat disagree
Master's degree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat disagree,⋯,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree
Doctoral degree,Neither agree nor disagree,Somewhat disagree,Somewhat disagree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Somewhat agree,Somewhat disagree,⋯,Somewhat disagree,Somewhat agree,Somewhat disagree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree
Doctoral degree,Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Strongly agree,Neither agree nor disagree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,⋯,Strongly disagree,Strongly disagree,Somewhat disagree,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree,Strongly disagree
Bachelor's degree in college (4-year),Somewhat agree,Neither agree nor disagree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Neither agree nor disagree,⋯,Somewhat agree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Somewhat agree,Somewhat agree,Neither agree nor disagree,Somewhat agree,Somewhat agree,Somewhat agree
Doctoral degree,Neither agree nor disagree,Somewhat agree,Somewhat disagree,Somewhat agree,Somewhat agree,Neither agree nor disagree,Strongly disagree,Somewhat disagree,Somewhat disagree,⋯,Strongly disagree,Neither agree nor disagree,Somewhat disagree,Strongly disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat disagree
Doctoral degree,Somewhat agree,Neither agree nor disagree,Somewhat agree,Neither agree nor disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,⋯,Somewhat agree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat disagree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat agree


In [ ]:
hum = datFull $>$
    select()